Cite this paper if this code helps you:
F. Satter and S. M. Masudul Ahsan, "Open Vocabulary Word Recognition From Transcribed Bangla Texts," 2023 26th International Conference on Computer and Information Technology (ICCIT), Cox's Bazar, Bangladesh, 2023, pp. 1-6, doi: 10.1109/ICCIT60459.2023.10441393.

Tutorial for Training: https://www.youtube.com/watch?v=amURyS6CAaY

Everything is same except


*   The third cell[models template]
*   faster_rcnn_pipeline.config file
*   "Creation of Label Map" cell





Necessary Dataset can be found here : https://data.mendeley.com/datasets/fnw59h7y89/2 in the "PromitoLipi2.1" folder

In [2]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf

In [ ]:
E#mount drive

from google.colab import drive
drive.mount('/content/gdrive')

# # this creates a symbolic link so that now the path /content/gdrive/My Drive/ is equal to /mydrive

# !ln -s /content/gdrive/My Drive/ /mydrive
# !ls /mydrive

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/models (1)/research

!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
#ekhane ekta . deya lagse

/content/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

2023-03-02 20:42:09.560806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 20:42:09.560923: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 20:42:09.560944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Running tests under Python 3.8.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2023-03-02 20:42:16.377904: W tensorflow/core/commo

In [ ]:
cd /content/gdrive/MyDrive/customTF2/data

/content/gdrive/.shortcut-targets-by-id/1sWxgb3JhaOFJans40zyiX8hXhfODE9Hx/customTF2/data


In [ ]:
#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name)
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
Successfully created label_map.pbtxt 


Creation of Label Map

In [ ]:
labels = [{'id':'1', 'name':1},
{'id':'2', 'name':2},
{'id':'3', 'name':3},
{'id':'4', 'name':4},
{'id':'5', 'name':5},
{'id':'6', 'name':6},
{'id':'7', 'name':7},
{'id':'8', 'name':8},
{'id':'9', 'name':9},
{'id':'10', 'name':10},
{'id':'11', 'name':11},
{'id':'12', 'name':12},
{'id':'13', 'name':13},
{'id':'14', 'name':14},
{'id':'15', 'name':15},
{'id':'16', 'name':16},
{'id':'17', 'name':17},
{'id':'18', 'name':18},
{'id':'19', 'name':19},
{'id':'20', 'name':20},
{'id':'21', 'name':21},
{'id':'22', 'name':22},
{'id':'23', 'name':23},
{'id':'24', 'name':24},
{'id':'25', 'name':25},
{'id':'26', 'name':26},
{'id':'27', 'name':27},
{'id':'28', 'name':28},
{'id':'29', 'name':29},
{'id':'30', 'name':30},
{'id':'31', 'name':31},
{'id':'32', 'name':32},
{'id':'33', 'name':33},
{'id':'34', 'name':34},
{'id':'35', 'name':35},
{'id':'36', 'name':36},
{'id':'37', 'name':37},
{'id':'38', 'name':38},
{'id':'39', 'name':39},
{'id':'40', 'name':40},
{'id':'41', 'name':41},
{'id':'42', 'name':42},
{'id':'43', 'name':43},
{'id':'44', 'name':44},
{'id':'45', 'name':45},
{'id':'46', 'name':46},
{'id':'47', 'name':47},
{'id':'48', 'name':48},
{'id':'49', 'name':49},
{'id':'50', 'name':50},
{'id':'51', 'name':51},
{'id':'52', 'name':52},
{'id':'53', 'name':53},
{'id':'54', 'name':54},
{'id':'55', 'name':55},
{'id':'56', 'name':56},
{'id':'57', 'name':57},
{'id':'58', 'name':58},
{'id':'59', 'name':59},
{'id':'60', 'name':60},
{'id':'61', 'name':61},
{'id':'62', 'name':62},
{'id':'63', 'name':63},
{'id':'64', 'name':64},
{'id':'65', 'name':65},
{'id':'66', 'name':66},
{'id':'67', 'name':67},
{'id':'68', 'name':68},
{'id':'69', 'name':69},
{'id':'70', 'name':70},
{'id':'71', 'name':71},
{'id':'72', 'name':72},
{'id':'73', 'name':73},
{'id':'74', 'name':74},
{'id':'75', 'name':75},
{'id':'76', 'name':76},
{'id':'77', 'name':77},
{'id':'78', 'name':78},
{'id':'79', 'name':79},
{'id':'80', 'name':80},
{'id':'81', 'name':81},
{'id':'82', 'name':82},
{'id':'83', 'name':83},
{'id':'84', 'name':84},
{'id':'85', 'name':85},
{'id':'86', 'name':86},
{'id':'87', 'name':87},
{'id':'88', 'name':88},
{'id':'89', 'name':89},
{'id':'90', 'name':90},
{'id':'91', 'name':91},
{'id':'92', 'name':92},]

with open('/content/gdrive/MyDrive/customTF2/data/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tid:{}\n'.format(label['id']))
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('}\n')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Usage:
#!python generate_tfrecord.py output.csv output_pb.txt /path/to/images output.tfrecords

#For train.record
!python /content/gdrive/MyDrive/customTF2/generate_tfrecord.py train_labels.csv  label_map.pbtxt images/ train.record

#For test.record
!python /content/gdrive/MyDrive/customTF2/generate_tfrecord.py test_labels.csv  label_map.pbtxt images/ test.record

2023-02-07 20:56:37.372998: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-07 20:56:37.373114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-07 20:56:37.373137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
groups: 100% 3210/3211 [18:19<00:00,  2.92it/s]
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/customTF2/generate_tfrecord.py", line 137, in <modul

In [ ]:
# Download the pre-trained model ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz into the data folder & unzip it

# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# !tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz

!tar -xzvf faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz


--2023-02-01 16:26:37--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.194.128, 2404:6800:4003:c04::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.194.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447961769 (427M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz’

faster_rcnn_incepti 100%[===================>] 427.21M  44.8MB/s    in 11s     

2023-02-01 16:26:48 (40.6 MB/s) - ‘faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz’ saved [447961769/447961769]

faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/
faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/checkpoint/
faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/gdrive/MyDrive/customTF2/training'

<IPython.core.display.Javascript object>

In [ ]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


TRAIN

In [ ]:
# Run the command below from the content/models/research/object_detection directory

"""
PIPELINE_CONFIG_PATH=path/to/pipeline.config
MODEL_DIR=path to training checkpoints directory
NUM_TRAIN_STEPS=50000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1

python model_main_tf2.py -- \
--model_dir=$MODEL_DIR --num_train_steps=$NUM_TRAIN_STEPS \
--sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
--pipeline_config_path=$PIPELINE_CONFIG_PATH \
--alsologtostderr
"""

!python model_main_tf2.py --pipeline_config_path=/content/gdrive/MyDrive/customTF2/data/faster_rcnn_pipeline.config --model_dir=/content/gdrive/MyDrive/customTF2/training  --alsologtostderr

2023-03-02 20:43:32.611251: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 20:43:32.612121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-02 20:43:32.612162: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-02 20:43:39.999374: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

In [ ]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
!python exporter_main_v2.py --trained_checkpoint_dir=/content/gdrive/MyDrive/customTF2/training --pipeline_config_path=/content/gdrive/MyDrive/customTF2/data/faster_rcnn_pipeline.config --output_directory /content/gdrive/MyDrive/customTF2/data/inference_graph

2023-02-08 20:45:22.524433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-08 20:45:22.524586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-08 20:45:22.524612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-08 20:45:25.478703: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

ACCURACY

In [ ]:
!python model_main_tf2.py --pipeline_config_path=/content/gdrive/MyDrive/customTF2/data/faster_rcnn_pipeline.config --model_dir=/content/gdrive/MyDrive/customTF2/training --alsologtostderr --checkpoint_dir=/content/gdrive/MyDrive/customTF2/training/

2023-02-05 22:29:44.866988: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0205 22:29:44.873000 140282786868288 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0205 22:29:44.873305 140282786868288 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 22:29:44.873412 140282786868288 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0205 22:29:44.873508 140282786868288 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0205 22:29:44.873672 140282786868288 model_lib_v2.py:1106] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:Reading unwe

In [ ]:
# Different font-type and font-size for labels text

!wget https://freefontsdownload.net/download/160187/arial.zip
!unzip arial.zip -d .

%cd utils/
!sed -i "s/font = ImageFont.truetype('arial.ttf', 24)/font = ImageFont.truetype('arial.ttf', 50)/" visualization_utils.py
%cd ..

--2023-01-30 18:46:01--  https://freefontsdownload.net/download/160187/arial.zip
Resolving freefontsdownload.net (freefontsdownload.net)... 172.67.180.27, 104.21.75.182, 2606:4700:3037::ac43:b41b, ...
Connecting to freefontsdownload.net (freefontsdownload.net)|172.67.180.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172821 (169K) [application/force-download]
Saving to: ‘arial.zip’

arial.zip           100%[===================>] 168.77K   270KB/s    in 0.6s    

2023-01-30 18:46:03 (270 KB/s) - ‘arial.zip’ saved [172821/172821]

Archive:  arial.zip
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@#################################@
@#        Downloaded from        #@
@#                               #@
@#   www.FreeFontsDownload.net   #@
@#                               #@
@# ---------- More site -------- #@
@#      https://funnytv.net      #@
@#                               #@
@#                               #@
@#################################@
@@@@@@@@@@@@@@@@@@@